In [1]:
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing,feature_selection
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

In [2]:
from sklearn import feature_selection
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [62]:
review1=pd.read_csv('Review_project_sentiment_wUser.csv')

In [63]:
## Tf-Idf (advanced variant of BoW)
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=5000, ngram_range=(1,1))

In [64]:
## split dataset
dtf_train, dtf_test = model_selection.train_test_split(review1, test_size=0.3)
## get target
y_train = dtf_train["useful_level"].values
y_test = np.array(dtf_test["useful_level"].values)

In [65]:
corpus1 = dtf_train["text_clean"]
vectorizer.fit(corpus1)
X_train = vectorizer.transform(corpus1)
dic_vocabulary = vectorizer.vocabulary_

In [66]:
y = dtf_train["useful_level"]
X_names = vectorizer.get_feature_names()
p_value_limit = 0.95
dtf_features = pd.DataFrame()
for cat in np.unique(y):
    chi2, p = feature_selection.chi2(X_train, y==cat)
    dtf_features = dtf_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    dtf_features = dtf_features.sort_values(["y","score"], 
                    ascending=[True,False])
    dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
X_names = dtf_features["feature"].unique().tolist()

In [67]:
for cat in np.unique(y):
    print("# {}:".format(cat))
    print("  . selected features:",
         len(dtf_features[dtf_features["y"]==cat]))
    print("  . top features:", ",".join(
dtf_features[dtf_features["y"]==cat]["feature"].values[:10]))
    print(" ")

# 0:
  . selected features: 1536
  . top features: company,food,great,amazing,atmosphere,call,told,see,review,month
 
# 1:
  . selected features: 547
  . top features: great,food,car,apartment,said,office,told,call,dog,company
 
# 2:
  . selected features: 462
  . top features: ride,park,ron,disney,food,tattoo,great,contract,mall,store
 


In [68]:
# refit vecorizor
vectorizer = feature_extraction.text.TfidfVectorizer(vocabulary=X_names)
vectorizer.fit(corpus1)
X_train = vectorizer.transform(corpus1)
dic_vocabulary = vectorizer.vocabulary_

In [69]:
corpus2 = dtf_test["text_clean"]
X_test= vectorizer.transform(corpus2)

In [87]:
dtf_train.shape

(38630, 21)

In [88]:
dtf_test.shape

(16557, 21)

In [71]:
AllotherTrain = dtf_train[["useful_level", "read_ease", "polarity", "subjectivity", 
                           "Adj_ratio",'review_count','NumElite','average_stars','NumofWords']]

In [89]:
X_train.shape

(38630, 1629)

In [90]:
X_test.shape

(16557, 1629)

In [72]:
EmbeddingTrain = pd.DataFrame(X_train.toarray(), columns = vectorizer.get_feature_names())

In [73]:
for i in ["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio",'review_count','NumElite','average_stars','NumofWords']:
    EmbeddingTrain[i] = AllotherTrain[i].values

In [74]:
EmbeddingTrain

,company,food,great,amazing,atmosphere,call,told,see,review,month,...,forest,useful_level,read_ease,polarity,subjectivity,Adj_ratio,review_count,NumElite,average_stars,NumofWords
0,0.000000,0.161856,0.000000,0.00000,0.100269,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,2,64.58,0.244345,0.554992,0.202312,1303,9,4.15,325
1,0.000000,0.156125,0.077486,0.00000,0.000000,0.000000,0.000000,0.103274,0.000000,0.000000,...,0.0,1,39.00,0.095000,0.418333,0.220339,16,0,4.19,114
2,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0,57.27,-0.483333,0.666667,0.444444,3,0,2.75,14
3,0.000000,0.037261,0.036985,0.00000,0.000000,0.057615,0.000000,0.049294,0.051002,0.059816,...,0.0,2,-160.80,0.055780,0.508000,0.243137,824,8,4.25,491
4,0.000000,0.000000,0.035570,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1,67.18,0.321462,0.588639,0.170854,21,0,4.21,356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38625,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0,89.58,0.266667,0.575000,0.250000,395,3,4.01,31
38626,0.000000,0.043796,0.000000,0.13306,0.000000,0.000000,0.000000,0.057941,0.000000,0.000000,...,0.0,2,53.17,0.254134,0.620768,0.165138,153,2,3.87,210
38627,0.000000,0.000000,0.000000,0.00000,0.000000,0.017256,0.047130,0.000000,0.000000,0.053745,...,0.0,1,71.58,0.060192,0.417879,0.129956,14,0,3.29,898
38628,0.000000,0.056365,0.000000,0.00000,0.000000,0.000000,0.317396,0.074568,0.000000,0.000000,...,0.0,0,-43.22,0.194866,0.457589,0.151515,7,0,4.29,276


In [75]:
NewXTrain = EmbeddingTrain.loc[:, EmbeddingTrain.columns != 'useful_level']

In [97]:
NewXTrain.shape

(38630, 1637)

In [78]:
NewYTrain = EmbeddingTrain.loc[:, EmbeddingTrain.columns == 'useful_level']

In [86]:
EmbeddingTrain.shape

(38630, 1638)

In [79]:
# test set
AllotherTest = dtf_test[["useful_level", "read_ease", "polarity", "subjectivity", 
                           "Adj_ratio",'review_count','NumElite','average_stars','NumofWords']]

In [80]:
EmbeddingTest = pd.DataFrame(X_test.toarray(), columns = vectorizer.get_feature_names())

In [92]:
EmbeddingTest.shape

(16557, 1638)

In [96]:
NewXTest.shape

(16557, 1637)

In [91]:
for i in ["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio",'review_count','NumElite','average_stars','NumofWords']:
    EmbeddingTest[i] = AllotherTest[i].values

In [95]:
NewXTest = EmbeddingTest.loc[:, EmbeddingTest.columns != 'useful_level']
NewYTest = EmbeddingTest.loc[:, EmbeddingTest.columns == 'useful_level'].values

In [23]:
from sklearn.metrics import mean_squared_error

# importing machine learning models for prediction
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(NewXTrain, NewYTrain, test_size=0.20)

In [99]:
X_senti_train=X_train[[ "read_ease", "polarity", "subjectivity", 
                         "Adj_ratio",'review_count','NumElite','average_stars','NumofWords']]

In [100]:
X_tfidf_train=X_train.drop(["read_ease", "polarity", "subjectivity", "Adj_ratio",
                           'review_count','NumElite','average_stars','NumofWords'], axis=1)

In [101]:
X_senti_valid=X_valid[[ "read_ease", "polarity", "subjectivity", 
                         "Adj_ratio",'review_count','NumElite','average_stars','NumofWords']]

In [102]:
X_tfidf_valid=X_valid.drop(["read_ease", "polarity", "subjectivity", "Adj_ratio",
                           'review_count','NumElite','average_stars','NumofWords'], axis=1)

In [37]:
NewXTest

,amazing,company,food,great,call,told,atmosphere,delicious,doctor,review,...,doughnut,splash,distancing,joining,nearby,christian,walt,jalapeños,castle,goal
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.151594,0.000000,0.0,0.0,0.0,0.110247,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.074698,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.098653,0.098090,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.239783,0.000000,0.0,0.0,0.0,0.174382,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16552,0.0,0.0,0.140102,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16553,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16554,0.0,0.0,0.000000,0.200112,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16555,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
X_senti_test=EmbeddingTest[[ "read_ease", "polarity", "subjectivity", 
                         "Adj_ratio",'review_count','NumElite','average_stars','NumofWords']]

In [104]:
X_tfidf_test=EmbeddingTest.drop(["read_ease", "polarity", "subjectivity", "Adj_ratio",
                           'review_count','NumElite','average_stars','NumofWords','useful_level'], axis=1)

In [105]:
model_1 = LinearRegression()
model_2 = xgb.XGBRegressor()

In [106]:
model_1.fit(X_senti_train, y_train)
val_pred_1 = model_1.predict(X_senti_valid)
test_pred_1 = model_1.predict(X_senti_test)

In [107]:
model_2.fit(X_tfidf_train, y_train)
val_pred_2 = model_2.predict(X_tfidf_valid)
test_pred_2 = model_2.predict(X_tfidf_test)

In [108]:
val_pred_1 = pd.DataFrame(val_pred_1)
test_pred_1 = pd.DataFrame(test_pred_1)

In [109]:
val_pred_2 = pd.DataFrame(val_pred_2)
test_pred_2 = pd.DataFrame(test_pred_2)

In [110]:
X_valid.index=list(range(7726))
NewXTest.index=list(range(16557))

In [111]:
df_val = pd.concat([X_valid, val_pred_1, val_pred_2], axis=1)
df_test = pd.concat([NewXTest, test_pred_1, test_pred_2], axis=1)

In [112]:
from sklearn.linear_model import LogisticRegression
final_model = LogisticRegression()
final_model.fit(df_val, y_valid)

C:\Users\Fannie\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Fannie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [113]:
final_pred = final_model.predict(df_test)

In [114]:
print(mean_squared_error(y_test, final_pred))

0.5712991483964487
